In [2]:
import numpy as np
import pandas as pd
import math
from datetime import datetime

import seaborn as sn
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/free-to-learn/Machine-Learning-DataSet/master/Data/00%20df.csv')
df.head()

,hours_per_week_bin,occupation_bin,msr_bin,capital_gl_bin,race_sex_bin,education_num_bin,education_bin,workclass_bin,age_bin,flag,y
0,b. 31-40,b. Mid - Low,b. Mid,c. > 0,c. High,c. 13,c. Bachelors,b. income,d. 36-40 & 56-60,train,0
1,a. 0-30,e. High,c. High,a. = 0,c. High,c. 13,c. Bachelors,b. income,e. 40-55,train,0
2,b. 31-40,a. Low,b. Mid,a. = 0,c. High,b. 9-12,b. Mid,b. income,d. 36-40 & 56-60,train,0
3,b. 31-40,a. Low,c. High,a. = 0,b. Mid,a. 0-8,a. Low,b. income,e. 40-55,train,0
4,b. 31-40,e. High,c. High,a. = 0,a. Low,c. 13,c. Bachelors,b. income,b. 26-30 & 71-100,train,0


In [4]:
df.tail()

,hours_per_week_bin,occupation_bin,msr_bin,capital_gl_bin,race_sex_bin,education_num_bin,education_bin,workclass_bin,age_bin,flag,y
48837,b. 31-40,e. High,b. Mid,a. = 0,b. Mid,c. 13,c. Bachelors,b. income,d. 36-40 & 56-60,test,0
48838,b. 31-40,b. Mid - Low,a. Low,a. = 0,b. Mid,b. 9-12,b. Mid,a. no income,c. 31-35 & 61-70,test,0
48839,d. 41-50 & 61-70,e. High,c. High,a. = 0,c. High,c. 13,c. Bachelors,b. income,d. 36-40 & 56-60,test,0
48840,b. 31-40,b. Mid - Low,a. Low,c. > 0,c. High,c. 13,c. Bachelors,b. income,e. 40-55,test,0
48841,e. 51-60,e. High,c. High,a. = 0,c. High,c. 13,c. Bachelors,b. income,c. 31-35 & 61-70,test,1


In [5]:
train = df[df['flag'] == 'train']
test = df[df['flag'] == 'test']

In [6]:
train.shape

(32561, 11)

In [7]:
test.shape

(16281, 11)

In [8]:
df.columns

Index(['hours_per_week_bin', 'occupation_bin', 'msr_bin', 'capital_gl_bin',
       'race_sex_bin', 'education_num_bin', 'education_bin', 'workclass_bin',
       'age_bin', 'flag', 'y'],
      dtype='object')

In [9]:
cat_features = ['age_bin','capital_gl_bin','education_bin','hours_per_week_bin','msr_bin','occupation_bin','race_sex_bin']
y_train = train['y']

In [10]:
x_train = train[['age_bin','capital_gl_bin','education_bin','hours_per_week_bin','msr_bin','occupation_bin','race_sex_bin']]
x_train.head()

,age_bin,capital_gl_bin,education_bin,hours_per_week_bin,msr_bin,occupation_bin,race_sex_bin
0,d. 36-40 & 56-60,c. > 0,c. Bachelors,b. 31-40,b. Mid,b. Mid - Low,c. High
1,e. 40-55,a. = 0,c. Bachelors,a. 0-30,c. High,e. High,c. High
2,d. 36-40 & 56-60,a. = 0,b. Mid,b. 31-40,b. Mid,a. Low,c. High
3,e. 40-55,a. = 0,a. Low,b. 31-40,c. High,a. Low,b. Mid
4,b. 26-30 & 71-100,a. = 0,c. Bachelors,b. 31-40,c. High,e. High,a. Low


In [11]:
x_train = pd.get_dummies(x_train, columns = cat_features, drop_first = True)
x_train.head()

,age_bin_b. 26-30 & 71-100,age_bin_c. 31-35 & 61-70,age_bin_d. 36-40 & 56-60,age_bin_e. 40-55,capital_gl_bin_b. < 0,capital_gl_bin_c. > 0,education_bin_b. Mid,education_bin_c. Bachelors,education_bin_d. Masters,education_bin_e. High,...,hours_per_week_bin_d. 41-50 & 61-70,hours_per_week_bin_e. 51-60,msr_bin_b. Mid,msr_bin_c. High,occupation_bin_b. Mid - Low,occupation_bin_c. Mid - Mid,occupation_bin_d. Mid - High,occupation_bin_e. High,race_sex_bin_b. Mid,race_sex_bin_c. High
0,0,0,1,0,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,1
1,0,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
2,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0


In [12]:
y_test = test['y']
x_test = test[['age_bin','capital_gl_bin','education_bin','hours_per_week_bin','msr_bin','occupation_bin','race_sex_bin']]
x_test.head()

,age_bin,capital_gl_bin,education_bin,hours_per_week_bin,msr_bin,occupation_bin,race_sex_bin
32561,a. 0-25,a. = 0,a. Low,b. 31-40,a. Low,b. Mid - Low,b. Mid
32562,d. 36-40 & 56-60,a. = 0,b. Mid,d. 41-50 & 61-70,c. High,b. Mid - Low,c. High
32563,b. 26-30 & 71-100,a. = 0,b. Mid,b. 31-40,c. High,d. Mid - High,c. High
32564,e. 40-55,c. > 0,b. Mid,b. 31-40,c. High,b. Mid - Low,b. Mid
32565,a. 0-25,a. = 0,b. Mid,a. 0-30,a. Low,b. Mid - Low,b. Mid


In [13]:
x_test = pd.get_dummies(x_test, columns = cat_features, drop_first = True)
x_test.head()

,age_bin_b. 26-30 & 71-100,age_bin_c. 31-35 & 61-70,age_bin_d. 36-40 & 56-60,age_bin_e. 40-55,capital_gl_bin_b. < 0,capital_gl_bin_c. > 0,education_bin_b. Mid,education_bin_c. Bachelors,education_bin_d. Masters,education_bin_e. High,...,hours_per_week_bin_d. 41-50 & 61-70,hours_per_week_bin_e. 51-60,msr_bin_b. Mid,msr_bin_c. High,occupation_bin_b. Mid - Low,occupation_bin_c. Mid - Mid,occupation_bin_d. Mid - High,occupation_bin_e. High,race_sex_bin_b. Mid,race_sex_bin_c. High
32561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
32562,0,0,1,0,0,0,1,0,0,0,...,1,0,0,1,1,0,0,0,0,1
32563,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1
32564,0,0,0,1,0,1,1,0,0,0,...,0,0,0,1,1,0,0,0,1,0
32565,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [14]:
sgd = SGDClassifier(loss = 'modified_huber', shuffle = True, random_state = 101)
sgd.fit(x_train, y_train)
sgd

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=101, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [15]:
y_pred = sgd.predict(x_test)
y_pred[0:5]

array([0, 0, 0, 1, 0], dtype=int64)

In [20]:
test_cal = pd.concat([pd.DataFrame(y_test).reset_index(drop = True), pd.DataFrame(y_pred).reset_index(drop=True)], axis = 1)
test_cal.rename(columns = {0 : 'New_Predicted'}, inplace = True)
test_cal

,y,New_Predicted
0,0,0
1,0,0
2,1,0
3,1,1
4,0,0
...,...,...
16276,0,0
16277,0,0
16278,0,1
16279,0,0


In [17]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89     12435
           1       0.67      0.62      0.64      3846

    accuracy                           0.84     16281
   macro avg       0.78      0.76      0.77     16281
weighted avg       0.83      0.84      0.84     16281



In [18]:
confusion_matrix(y_test, y_pred)

array([[11244,  1191],
       [ 1456,  2390]], dtype=int64)

In [19]:
accuracy_score(y_test, y_pred)

0.8374178490264725